In [ ]:
!pip install geopandas

In [ ]:
import pandas as pd
from glob import glob
import os
import numpy as np
from tqdm import tqdm, tqdm_notebook
import json
import geopandas as gpd

In [ ]:
df = gpd.read_file('/content/drive/MyDrive/LH/raw data/10.김해시_성연령별_거주인구격자.geojson')

In [ ]:
np.mean(df['2018_m_0g_pop'])

0.9915321118529105

In [ ]:
df['2020_유소년'] = df['2020_m_0g_pop']+df['2020_w_0g_pop']
df['2020_생산가능'] = df['2020_m_20g_pop'] + df['2020_w_20g_pop']+df['2020_m_30g_pop'] + df['2020_w_30g_pop']+df['2020_m_40g_pop'] + df['2020_w_40g_pop']+df['2020_m_50g_pop'] + df['2020_w_50g_pop']+df['2020_m_60g_pop'] + df['2020_w_60g_pop']
df['2020_고령'] = df['2020_m_70g_pop']+df['2020_w_70g_pop']
df['2020_전체거주인구'] = df['2020_유소년']+df['2020_생산가능']+df['2020_고령']

In [ ]:
df['2021_유소년'] = df['2021_m_0g_pop']+df['2021_w_0g_pop']
df['2021_생산가능'] = df['2021_m_20g_pop'] + df['2021_w_20g_pop']+df['2021_m_30g_pop'] + df['2021_w_30g_pop']+df['2021_m_40g_pop'] + df['2021_w_40g_pop']+df['2021_m_50g_pop'] + df['2021_w_50g_pop']+df['2021_m_60g_pop'] + df['2021_w_60g_pop']
df['2021_고령'] = df['2021_m_70g_pop']+df['2021_w_70g_pop']
df['2021_전체거주인구'] =df['2021_유소년']+df['2021_생산가능']+df['2021_고령']

In [ ]:
df = df[['gid','geometry','2020_유소년','2020_생산가능','2020_고령','2020_전체거주인구','2021_유소년','2021_생산가능','2021_고령','2021_전체거주인구']]

In [ ]:
df

In [ ]:
df['2020_부양인구지수'] = (df['2020_유소년']+df['2020_고령'])/df['2020_전체거주인구']
df['2021_부양인구지수'] = (df['2021_유소년']+df['2021_고령'])/df['2021_전체거주인구']

/usr/local/lib/python3.7/dist-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [ ]:
new = df[['gid','geometry','2020_부양인구지수','2021_부양인구지수']]
new = new.fillna(0)
new

,gid,geometry,2020_부양인구지수,2021_부양인구지수
0,마라094999,"POLYGON ((128.70309 35.28863, 128.70311 35.289...",0.0,0.0
1,마라095991,"POLYGON ((128.70409 35.28140, 128.70410 35.282...",0.0,0.0
2,마라095992,"POLYGON ((128.70410 35.28231, 128.70411 35.283...",0.0,0.0
3,마라095993,"POLYGON ((128.70411 35.28321, 128.70413 35.284...",0.0,0.0
4,마라095994,"POLYGON ((128.70413 35.28411, 128.70414 35.285...",0.0,0.0
...,...,...,...,...
46878,마마363001,"POLYGON ((128.99887 35.28713, 128.99889 35.288...",0.0,0.0
46879,마마363002,"POLYGON ((128.99889 35.28803, 128.99890 35.288...",0.0,0.0
46880,마마364000,"POLYGON ((128.99995 35.28621, 128.99997 35.287...",0.0,0.0
46881,마마364001,"POLYGON ((128.99997 35.28711, 128.99998 35.288...",0.0,0.0


In [ ]:
from sklearn.preprocessing import  MinMaxScaler
scaler = MinMaxScaler()
scaled1= scaler.fit_transform(new[['2020_부양인구지수']].values) * 100
scaled2= scaler.fit_transform(new[['2021_부양인구지수']].values) * 100
df = pd.DataFrame(scaled1)
df2= pd.DataFrame(scaled2)
df3= pd.concat([df,df2],axis=1)

In [ ]:
df3.columns = ['2020_부양인구지수','2021_부양인구지수']

In [ ]:
total = pd.concat([new[['gid','geometry']],df3],axis=1)

In [ ]:
total.to_csv('/content/drive/MyDrive/LH/complete data/격자별_부양인구지수.csv', index=False)

## 외국인위험지수

In [ ]:
foreign_df = gpd.read_file('/content/drive/MyDrive/LH/raw data/11.김해시_외국인 읍면동별 격자.geojson')

In [ ]:
tqdm.pandas()
foreign_df['2020_생산가능']  = foreign_df['gid'].progress_apply(lambda x : df.query("gid == '" + x + "' ")['2020_생산가능'].iloc[0] )

100%|██████████| 46883/46883 [04:18<00:00, 181.68it/s]


In [ ]:
foreign_df['2021_생산가능']  = foreign_df['gid'].progress_apply(lambda x : df.query("gid == '" + x + "' ")['2021_생산가능'].iloc[0])

100%|██████████| 46883/46883 [04:11<00:00, 186.55it/s]


In [ ]:
foreign_df['2020_foreigner_pop'] = foreign_df['2020_foreigner_pop'].astype(float)
foreign_df['2021_foreigner_pop'] = foreign_df['2021_foreigner_pop'].astype(float)
foreign_df = foreign_df.fillna(0)

In [ ]:
foreign_df['2020_외국인위험지수'] = foreign_df['2020_foreigner_pop']/foreign_df['2020_생산가능']
foreign_df['2021_외국인위험지수'] = foreign_df['2021_foreigner_pop']/foreign_df['2021_생산가능']
foreign_df = foreign_df.fillna(0)

In [ ]:
foreign_new = foreign_df[['gid','geometry','2020_외국인위험지수','2021_외국인위험지수']]
foreign_new = foreign_new.replace([np.inf, -np.inf], np.nan)
foreign_new = foreign_new.fillna(0)

In [ ]:
foreign_new

,gid,geometry,2020_외국인위험지수,2021_외국인위험지수
0,마라231909,"MULTIPOLYGON (((128.85237 35.20590, 128.85239 ...",0.0,0.0
1,마라154986,"MULTIPOLYGON (((128.76888 35.27623, 128.76889 ...",0.0,0.0
2,마라213944,"MULTIPOLYGON (((128.83312 35.23767, 128.83313 ...",0.0,0.0
3,마마105025,"MULTIPOLYGON (((128.71554 35.31194, 128.71555 ...",0.0,0.0
4,마라202940,"MULTIPOLYGON (((128.82097 35.23420, 128.82099 ...",0.0,0.0
...,...,...,...,...
46878,마마175064,"MULTIPOLYGON (((128.79308 35.34630, 128.79310 ...",0.0,0.0
46879,마마210038,"MULTIPOLYGON (((128.83121 35.32245, 128.83122 ...",0.0,0.0
46880,마라130867,"MULTIPOLYGON (((128.74086 35.16923, 128.74087 ...",0.0,0.0
46881,마라166953,"MULTIPOLYGON (((128.78160 35.24635, 128.78162 ...",0.0,0.0


In [ ]:
from sklearn.preprocessing import  MinMaxScaler
scaler = MinMaxScaler()
scaled1= scaler.fit_transform(foreign_new[['2020_외국인위험지수']].values) * 100
scaled2= scaler.fit_transform(foreign_new[['2021_외국인위험지수']].values) * 100
df = pd.DataFrame(scaled1)
df2= pd.DataFrame(scaled2)
df3= pd.concat([df,df2],axis=1)

In [ ]:
df3.columns = ['2020_외국인위험지수','2021_외국인위험지수']

In [ ]:
total = pd.concat([foreign_new[['gid','geometry']],df3],axis=1)

In [ ]:
total.to_csv('/content/drive/MyDrive/LH/complete data/격자별_외국인위험지수.csv', index=False)